In [181]:
'''
Logistical Regretion
f = wx + b, sigmoid at end
0) Prepare data
1) Design Model (input, output, forward pass)
2) Construct loss and optimizer
3) Training loop
    - Forward pass: compute prediction
    - Backward pass: gradients
    - Update weights and empty gradients
'''
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [182]:
# Prep data
bc = datasets.load_breast_cancer()

X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(f'n_samples: {n_samples}; n_features: {n_features}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6969)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# make tensors and convert to float
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape y tensors to column vectors
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

n_samples: 569; n_features: 30


In [183]:
# Design Model
class LogisticRegresion(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegresion, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegresion(n_features)

In [184]:
# Construct loss & Optimizer
learning_rate = 0.01
criterion = nn.BCELoss() # Binary Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# (model.parameters(), lr=learning_rate)

In [185]:
# Training Loop
n = 1000
for epoch in range(n):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    # backward pass
    loss.backward()
    # update wieghts
    optimizer.step()
    optimizer.zero_grad()

    if (epoch+1) % (n/10) == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')



epoch: 100, loss = 0.2565
epoch: 200, loss = 0.1854
epoch: 300, loss = 0.1540
epoch: 400, loss = 0.1354
epoch: 500, loss = 0.1229
epoch: 600, loss = 0.1136
epoch: 700, loss = 0.1065
epoch: 800, loss = 0.1008
epoch: 900, loss = 0.0961
epoch: 1000, loss = 0.0922


In [186]:
with torch.no_grad(): # evaluation done without gradient
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9561
